In [1]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
import numpy as np
import os, json, cv2, random
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

In [2]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("my_dataset_train", {}, r"D:/HaxS/Dataset/Car dentss.v1i.coco-segmentation/train/_annotations.coco.json", r"D:/HaxS/Dataset/Car dentss.v1i.coco-segmentation/train")
register_coco_instances("my_dataset_val", {}, r"D:/HaxS/Dataset/Car dentss.v1i.coco-segmentation/valid/_annotations.coco.json", r"D:/HaxS/Dataset/Car dentss.v1i.coco-segmentation/valid")

In [3]:
train_metadata = MetadataCatalog.get("my_dataset_train")
train_dataset_dicts = DatasetCatalog.get("my_dataset_train")
val_metadata = MetadataCatalog.get("my_dataset_val")
val_dataset_dicts = DatasetCatalog.get("my_dataset_val")

WARNING [09/12 16:38:04 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[09/12 16:38:04 d2.data.datasets.coco]: Loaded 2478 images in COCO format from D:/HaxS/Dataset/Car dentss.v1i.coco-segmentation/train/_annotations.coco.json
WARNING [09/12 16:38:04 d2.data.datasets.coco]: 
Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.

[09/12 16:38:04 d2.data.datasets.coco]: Loaded 101 images in COCO format from D:/HaxS/Dataset/Car dentss.v1i.coco-segmentation/valid/_annotations.coco.json


In [4]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.OUTPUT_DIR = "D:\HaxS\Models\Medium"
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)

cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  # We have 4 classes.

cfg.MODEL.WEIGHTS=os.path.join(cfg.OUTPUT_DIR,"model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6 # Custom
predictor = DefaultPredictor(cfg)

[09/12 16:38:04 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from D:\HaxS\Models\Medium\model_final.pth ...


C:\Users\MUJ\conda3\envs\detectron_env\lib\site-packages\fvcore\common\checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=tor

In [ ]:
dir_path = r"D:\HaxS\Dataset\Car dentss.v1i.coco-segmentation\valid"

files = [f for f in os.listdir(dir_path) if f.endswith(".jpg")]

random_file = random.choice(files)

file_path = os.path.join(dir_path, random_file)

new_im = cv2.imread(file_path)

outputs = predictor(new_im)

v=Visualizer(new_im[:, :, ::-1], metadata=train_metadata,instance_mode=ColorMode.SEGMENTATION)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))

cv2.imshow("Segmented Image", out.get_image()[:, :, ::-1])
cv2.waitKey(0)
cv2.destroyAllWindows()